# Baseline Model
 - Oriented_rcnn_r50_fpn_1x_dota_le90

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

In [ ]:
!pip install wandb -qU

In [ ]:
!pwd

In [ ]:
!pip install openmim

In [ ]:
!mim install mmcv-full===1.6.0

In [ ]:
!mim install mmdet

In [ ]:
!mim install mmengine

# Git clone part


In [ ]:
!git clone https://github.com/open-mmlab/mmrotate.git

In [ ]:
%cd mmrotate

In [ ]:
!pwd

In [ ]:
!pip install -r requirements/build.txt

In [ ]:
!pip install -e .

# Oriented_rcnn download

In [ ]:
!mim download mmrotate --config oriented_rcnn_r50_fpn_1x_dota_le90 --dest .

#Google Drive mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import wandb
wandb.login()

In [ ]:
import os
from zipfile import ZipFile
import time

# unzip dataset 
def extract_zip(zip_path,output_path):
  
  """
  Extract .zip File
  Args
          - zip_path(str) : The path of zip file  
  """
  os.makedirs(output_path, exist_ok=True)
  
  with ZipFile(zip_path) as zip:
      start_time = time.time()
      zip.extractall(output_path) 
      print
      print(zip_path,'Unzip Complete')
      print(zip_path," Unzip Time: {:.4f}sec".format((time.time() - start_time)))

In [ ]:
# train
dataset_path = '/content/drive/MyDrive/00.AIFFEL/03.해커톤/00.SIA 해커톤/00.잘할거SIA/data/'

train_img_path = os.path.join(dataset_path,'train_images(sub_all_type).zip')
train_ann_path = os.path.join(dataset_path,'train_labelTxt_main.zip')

output_path = '/content/mmrotate/data/split_ss_dota/'
os.makedirs(output_path, exist_ok=True)


train_output_path = '/content/mmrotate/data/split_ss_dota/train'


# unzip 
extract_zip(train_img_path,'/content/mmrotate/data/split_ss_dota/train/images')
extract_zip(train_ann_path,'/content/mmrotate/data/split_ss_dota/train/annfiles')

# os.rename('/content/mmrotate/data/split_ss_dota/train/train_annfiles','/content/mmrotate/data/split_ss_dota/train/annfiles')
# os.rename('/content/mmrotate/data/split_ss_dota/train/train_images','/content/mmrotate/data/split_ss_dota/train/images')

/content/drive/MyDrive/00.AIFFEL/03.해커톤/00.SIA 해커톤/00.잘할거SIA/data/train_images(sub_all_type).zip Unzip Complete
/content/drive/MyDrive/00.AIFFEL/03.해커톤/00.SIA 해커톤/00.잘할거SIA/data/train_images(sub_all_type).zip  Unzip Time: 577.5717sec
/content/drive/MyDrive/00.AIFFEL/03.해커톤/00.SIA 해커톤/00.잘할거SIA/data/train_labelTxt_main.zip Unzip Complete
/content/drive/MyDrive/00.AIFFEL/03.해커톤/00.SIA 해커톤/00.잘할거SIA/data/train_labelTxt_main.zip  Unzip Time: 1.3349sec


In [ ]:
# validation
dataset_path = '/content/drive/MyDrive/00.AIFFEL/03.해커톤/00.SIA 해커톤/00.잘할거SIA/data/'

validation_img_path = os.path.join(dataset_path,'val_images(sub_all_type).zip')
validation_ann_path = os.path.join(dataset_path,'val_labelTxt_main.zip')

output_path = '/content/mmrotate/data/split_ss_dota/'
os.makedirs(output_path, exist_ok=True)


val_output_path = '/content/mmrotate/data/split_ss_dota/val'


# unzip 
extract_zip(validation_img_path,val_output_path + '/images')
extract_zip(validation_ann_path,val_output_path + '/annfiles')

In [ ]:
# convert tif to png  (-> )

def convert_tif_to_png(img_dir):
    import os
    tif_files = os.listdir(img_dir)
    for tif_file in tif_files:
        png_file = tif_file.replace('.tif','.png')
        os.replace(f"{img_dir}/{tif_file}", f"{img_dir}/{png_file}")
    print(img_dir,': convert complete')

train_img_dir = '/content/mmrotate/data/split_ss_dota/train/images'
val_img_dir = '/content/mmrotate/data/split_ss_dota/val/images'

convert_tif_to_png(train_img_dir)
convert_tif_to_png(val_img_dir)

#Config file 

## Dataset

In [ ]:

########################## 새로운 데이터셋 정의 : FAIR1MDataset 클래스를 생성하여 fair1m.py 생성하기 ############################

FAIR1MDataset_text = '''

from .builder import ROTATED_DATASETS
from .dota import DOTADataset
import os
import os.path as osp
import xml.etree.ElementTree as ET
from collections import OrderedDict
import numpy as np
from PIL import Image
from mmrotate.core import eval_rbbox_map


@ROTATED_DATASETS.register_module()
class FAIR1MDataset(DOTADataset):
    """fair1m dataset for detection """
    CLASSES = ('Airplane','Ship', 'Vehicle')
    PALETTE = [(165, 42, 42), (189, 183, 107), (0, 255, 0)]

    def __init__(self, **kwargs):
        super(FAIR1MDataset, self).__init__(**kwargs)

    def evaluate(self,
                 results,
                 metric= ['mAP','F1Score'],
                 logger=None,
                 proposal_nums=(100, 300, 1000),
                 iou_thr=0.5,
                 scale_ranges=None,
                 nproc=4):

        """Evaluate the dataset.
        Args:
            results (list): Testing results of the dataset.
            metric (str | list[str]): Metrics to be evaluated.
            logger (logging.Logger | None | str): Logger used for printing
                related information during evaluation. Default: None.
            proposal_nums (Sequence[int]): Proposal number used for evaluating
                recalls, such as recall@100, recall@1000.
                Default: (100, 300, 1000).
            iou_thr (float | list[float]): IoU threshold. It must be a float
                when evaluating mAP, and can be a list when evaluating recall.
                Default: 0.5.
            scale_ranges (list[tuple] | None): Scale ranges for evaluating mAP.
                Default: None.
            use_07_metric (bool): Whether to use the voc07 metric.
            nproc (int): Processes used for computing TP and FP.
                Default: 4.
        """
        
        # super().evaluate()
        nproc = min(nproc, os.cpu_count())

        # metric 이 없으면 mAP,F1Score 쓰기
        if not isinstance(metric, str):
            assert len(metric) == 2
            metric = metric
        
        # allowed_metrics 외 metric을 적으면 해당 메트릭은 지원하지 않는다는 예외처리하기   
        allowed_metrics = ['mAP','F1Score']
        if metric not in allowed_metrics:
            raise KeyError(f'metric {metric} is not supported')

        annotations = [self.get_ann_info(i) for i in range(len(self))]
        eval_results = {}


        # mAP, F1Score metric 정의 (metric으로 mAP, F1Score 두개 다 썼을때 )
        if ('mAP' and 'F1Score') in metric  :
            assert isinstance(iou_thr, float)

            mean_ap, mean_f1, _ = eval_rbbox_map(
                results,
                annotations,
                scale_ranges=scale_ranges,
                iou_thr=iou_thr,
                dataset=self.CLASSES,
                logger=logger,
                nproc=nproc)
            eval_results['mAP'] = mean_ap
            eval_results['F1Score'] = mean_f1

        # mAP metric 정의 (mAP 일때는 iou별 AP 도 같이 계산해야함)
        if metric == 'mAP' :
            assert isinstance(iou_thr, float)
            mean_ap, _, _ = eval_rbbox_map(
                results,
                annotations,
                scale_ranges=scale_ranges,
                iou_thr=iou_thr,
                dataset=self.CLASSES,
                logger=logger,
                nproc=nproc)
            eval_results['mAP'] = mean_ap


        # F1Score metric 정의    
        elif metric == 'F1Score':
            assert isinstance(iou_thr, float)
            _, mean_f1, _ = eval_rbbox_map(
                results,
                annotations,
                scale_ranges=scale_ranges,
                iou_thr=iou_thr,
                dataset=self.CLASSES,
                logger=logger,
                nproc=nproc)
            eval_results['F1Score'] = mean_f1


        else:
            raise NotImplementedError

        return eval_results
        
'''

FAIR1MDataset_path = '/content/mmrotate/mmrotate/datasets/fair1m.py'

f = open(FAIR1MDataset_path, "w")
f.write(FAIR1MDataset_text)
f.close()

In [ ]:
FAIR1MDataset_init = '''
# Copyright (c) OpenMMLab. All rights reserved.
from .builder import build_dataset  # noqa: F401, F403
from .dota import DOTADataset  # noqa: F401, F403
from .hrsc import HRSCDataset  # noqa: F401, F403
from .pipelines import *  # noqa: F401, F403
from .sar import SARDataset  # noqa: F401, F403
from .fair1m import FAIR1MDataset

__all__ = ['SARDataset', 'FAIR1MDataset', 'DOTADataset', 'build_dataset', 'HRSCDataset']
'''

FAIR1MDataset_init_path = '/content/mmrotate/mmrotate/datasets/__init__.py'

f = open(FAIR1MDataset_init_path, "w")
f.write(FAIR1MDataset_init)
f.close()

## eval_map

In [ ]:
########## eval_map ############

eval_map = '''

# Copyright (c) OpenMMLab. All rights reserved.
from multiprocessing import get_context

import numpy as np
import torch
from mmcv.ops import box_iou_rotated
from mmcv.utils import print_log
from mmdet.core import average_precision
from terminaltables import AsciiTable

# tpfp 가려내는 함수 -> 밑에선 클래스 별로 계산함
def tpfp_default(det_bboxes,
                 gt_bboxes,
                 gt_bboxes_ignore=None,
                 iou_thr=0.5,
                 area_ranges=None):
    """Check if detected bboxes are true positive or false positive.

    Args:
        det_bboxes (ndarray): Detected bboxes of this image, of shape (m, 6).
        gt_bboxes (ndarray): GT bboxes of this image, of shape (n, 5).
        gt_bboxes_ignore (ndarray): Ignored gt bboxes of this image,
            of shape (k, 5). Default: None
        iou_thr (float): IoU threshold to be considered as matched.
            Default: 0.5.
        area_ranges (list[tuple] | None): Range of bbox areas to be evaluated,
            in the format [(min1, max1), (min2, max2), ...]. Default: None.

    Returns:
        tuple[np.ndarray]: (tp, fp) whose elements are 0 and 1. The shape of
            each array is (num_scales, m).
    """

    # an indicator of ignored gts
    det_bboxes = np.array(det_bboxes)
    gt_ignore_inds = np.concatenate(
        # gt_bboxes는 밑에서 결과에 따라 0,1로 바꾸는거고. gt_bboxes_ignore은 우선 ignore 이니까 다 1로 해놓음
        (np.zeros(gt_bboxes.shape[0], dtype=np.bool),
         np.ones(gt_bboxes_ignore.shape[0], dtype=np.bool)))
    # stack gt_bboxes and gt_bboxes_ignore for convenience
    gt_bboxes = np.vstack((gt_bboxes, gt_bboxes_ignore))

    num_dets = det_bboxes.shape[0]
    num_gts = gt_bboxes.shape[0]
    if area_ranges is None:
        area_ranges = [(None, None)]
    num_scales = len(area_ranges) # num_scales == 1 is None else 2???
    # tp and fp are of shape (num_scales, num_gts), each row is tp or fp of
    # a certain scale [[ 0, 이 num_dets개 ]]
    tp = np.zeros((num_scales, num_dets), dtype=np.float32)
    fp = np.zeros((num_scales, num_dets), dtype=np.float32)

####################### 이미지에 gt가 없음 ########################################
    # 이번 이미지에서 gt가 없으면 det전부를 fp로 변경
    if gt_bboxes.shape[0] == 0:
        if area_ranges == [(None, None)]:
            # fp 전체를 1로 바꿈
            fp[...] = 1
        else:
            raise NotImplementedError
        return tp, fp

######################## 이미지에 gt가 있음 #########################################
    # 이미지에 gt가 있는 상태에서 iou에따른 tp, fp 계산
    ious = box_iou_rotated(
        torch.from_numpy(det_bboxes).float(),
        torch.from_numpy(gt_bboxes).float()).numpy()

    # for each det, the max iou with all gts
    # 열 중에서(axis = 1) 큰 원소를 반환
    ious_max = ious.max(axis=1)
    # for each det, which gt overlaps most with it
    # 열 중에서(axis = 1) 큰 원소의 인덱스를 반환
    ious_argmax = ious.argmax(axis=1)
    # sort all dets in descending order by scores 스코어 제일 높은 값 sort_inds??
    sort_inds = np.argsort(-det_bboxes[:, -1]) # shape of det_bboxes : (m, 6)
    for k, (min_area, max_area) in enumerate(area_ranges):
        gt_covered = np.zeros(num_gts, dtype=bool)
        # area_ranges가 지정되지 않은 경우 gt_area_ignore는 모두 False입니다.
        if min_area is None:
            gt_area_ignore = np.zeros_like(gt_ignore_inds, dtype=bool)
        else:
            raise NotImplementedError
        for i in sort_inds:
            # 스코어 높은 값을 하나씩 꺼내서 그 값이 IOU thres 보다 높으면 그걸로 gt랑 매칭함
            if ious_max[i] >= iou_thr:
                matched_gt = ious_argmax[i]
                if not (gt_ignore_inds[matched_gt]
                        or gt_area_ignore[matched_gt]):
                    if not gt_covered[matched_gt]:
                        gt_covered[matched_gt] = True
                    # 매칭한 gt가 ignore이 아니고 gts에 있으면 tp / 아니면 fp
                        tp[k, i] = 1
                    else:
                        fp[k, i] = 1
                # otherwise ignore this detected bbox, tp = 0, fp = 0
            elif min_area is None:
                fp[k, i] = 1
            else:
                bbox = det_bboxes[i, :5]
                area = bbox[2] * bbox[3]
                if area >= min_area and area < max_area:
                    fp[k, i] = 1
    return tp, fp


# 디텍팅한 결과값을 가져와서 우리가 원하는 클래스에 있으면, 클래스 별로 
def get_cls_results(det_results, annotations, class_id):
    """Get det results and gt information of a certain class.

    Args:
        det_results (list[list]): 
            det_results (list[list]): [[cls1_det, cls2_det, ...], ...].
            The outer list indicates images, 
            and the inner list indicates per-class detected bboxes.
            => len(det_results) == 이미지 갯수, len(det_results[0]) == 클래스별로 감지된 bbox 갯수 
        annotations (list[dict]): Same as `eval_map()`.
        class_id (int): ID of a specific class.

    Returns:
        tuple[list[np.ndarray]]: detected bboxes, gt bboxes, ignored gt bboxes
    """
    # 디텍트 결과에서 이미지별 클래스 뽑아냄
    cls_dets = [img_res[class_id] for img_res in det_results]

    cls_gts = []
    cls_gts_ignore = []
    for ann in annotations:
        #어노테이션의 라벨이 클래스 id에 있으면 그 박스들을 cls_gt에 넣음
        gt_inds = ann['labels'] == class_id
        cls_gts.append(ann['bboxes'][gt_inds, :])
        # labels_ignore 값이 있으면 그 클래스 id는 cls_gts_ignore 값에 넣음
        if ann.get('labels_ignore', None) is not None:
            ignore_inds = ann['labels_ignore'] == class_id
            cls_gts_ignore.append(ann['bboxes_ignore'][ignore_inds, :])

        else:
            cls_gts_ignore.append(torch.zeros((0, 5), dtype=torch.float64))

    #cls_dets : 총 디텍트 결과, cls_gts : 디텍팅 결과에서 지정 클래스인것, cls_gts_ignore : 무시 값 
    return cls_dets, cls_gts, cls_gts_ignore


def eval_rbbox_map(det_results,
                   annotations,
                   scale_ranges=None,
                   iou_thr=0.5,
                   use_07_metric=True,
                   dataset=None,
                   logger=None,
                   nproc=4):
    """Evaluate mAP of a rotated dataset.

    Args:
        det_results (list[list]): 이미지 마다 디텍팅한 클래스 별 bbox
            [[cls1_det, cls2_det, ...], ...].
            The outer list indicates images, and the inner list indicates
            per-class detected bboxes.
        annotations (list[dict]): gt annotation an image
        Ground truth annotations where each item of the list indicates an image. 
        Keys of annotations are:
            - `bboxes`: numpy array of shape (n, 5) => n개가 5포인트로
            - `labels`: numpy array of shape (n, ) => n개의 클래스
            - `bboxes_ignore` (optional): numpy array of shape (k, 5) => 무시할 비박스
            - `labels_ignore` (optional): numpy array of shape (k, ) => 무시할 클래스

        scale_ranges (list[tuple] | None): 계산할 size min-max 지정
            Range of scales to be evaluated, in the format [(min1, max1), (min2, max2), ...]. 
            A range of (32, 64) means the area range between (32**2, 64**2).
            Default: None.

        iou_thr (float): IoU threshold to be considered as matched.
            Default: 0.5.

        use_07_metric (bool): Whether to use the voc07 metric.
        dataset (list[str] | str | None): Dataset name or dataset classes,
            there are minor differences in metrics for different datasets, e.g.
            "voc07", "imagenet_det", etc. Default: None.

        logger (logging.Logger | str | None): The way to print the mAP
            summary. See `mmcv.utils.print_log()` for details. Default: None.
        nproc (int): Processes used for computing TP and FP.
            Default: 4.

    Returns:
        tuple: (mAP, [dict, dict, ...])
    """
    #이미지수랑 어노테이션 수 안맞으면 오류!
    assert len(det_results) == len(annotations)


    num_imgs = len(det_results)
    num_scales = len(scale_ranges) if scale_ranges is not None else 1
    num_classes = len(det_results[0])  # positive class num // len(det_results[0]) == 클래스별로 감지된 bbox 갯수 
    area_ranges = ([(rg[0]**2, rg[1]**2) for rg in scale_ranges]
                   if scale_ranges is not None else None)

    pool = get_context('spawn').Pool(nproc)
    eval_results = []
    # 클래스 별로 계산
    for i in range(num_classes):
        # get gt and det bboxes of this class
        #cls_dets : 총 디텍트 결과, cls_gts : 디텍팅 결과에서 지정 클래스인것, cls_gts_ignore : 무시 값 
        cls_dets, cls_gts, cls_gts_ignore = get_cls_results(
            det_results, annotations, i)

        # compute tp and fp for each image with multiple processes
        tpfp = pool.starmap(
            tpfp_default,
            zip(cls_dets, cls_gts, cls_gts_ignore,
                [iou_thr for _ in range(num_imgs)],
                [area_ranges for _ in range(num_imgs)]))
        tp, fp = tuple(zip(*tpfp))

        # calculate gt number of each scale
        # ignored gts or gts beyond the specific scale are not counted
        # num_gts : area_ranges 없으면 디텍팅 결과에서 지정 클래스인 수, 있으면 사이즈에 맞는 수만 들어감
        num_gts = np.zeros(num_scales, dtype=int)
        for _, bbox in enumerate(cls_gts):
            if area_ranges is None:
                num_gts[0] += bbox.shape[0]
            else:
                gt_areas = bbox[:, 2] * bbox[:, 3]
                for k, (min_area, max_area) in enumerate(area_ranges):
                    num_gts[k] += np.sum((gt_areas >= min_area)
                                         & (gt_areas < max_area))
        # sort all det bboxes by score, also sort tp and fp
        cls_dets = np.vstack(cls_dets)
        num_dets = cls_dets.shape[0]
        sort_inds = np.argsort(-cls_dets[:, -1])
        tp = np.hstack(tp)[:, sort_inds]
        fp = np.hstack(fp)[:, sort_inds]

        # calculate recall and precision with tp and fp
        tp = np.cumsum(tp, axis=1)
        fp = np.cumsum(fp, axis=1)
        eps = np.finfo(np.float32).eps
        recalls = tp / np.maximum(num_gts[:, np.newaxis], eps) # num_gts가 gt수니까
        precisions = tp / np.maximum((tp + fp), eps)

        # calculate AP
        # scale_ranges 지정 되어 있으면 rec, prec, num_gts 값 변경
        if scale_ranges is None:
            recalls = recalls[0, :]
            precisions = precisions[0, :]
            num_gts = num_gts.item()
        mode = 'area' if not use_07_metric else '11points'
        ap = average_precision(recalls, precisions, mode)
        # calculate F1 추가
        f1_score = 2 * precisions * recalls / np.maximum((precisions+recalls),eps)
        f1_score = f1_score.mean()

        eval_results.append({
            'num_gts': num_gts,
            'num_dets': num_dets,
            'recall': recalls,
            'precision': precisions,
            'ap': ap,
            'F1' : f1_score
        })
    pool.close()

    # 클래스 별로 계산끝@ eval_results에 클래스별로 값 들어가있음
    
    # 만약 scale_ranges 가 지정되어 있으면,
    if scale_ranges is not None:
        # shape (num_classes, num_scales)
        ##### mF1 score 추가
        all_ap = np.vstack([cls_result['ap'] for cls_result in eval_results])
        all_f1 = np.vstack([cls_result['F1'] for cls_result in eval_results])
        all_num_gts = np.vstack(
            [cls_result['num_gts'] for cls_result in eval_results])
        mean_ap = []
        mean_f1 = []
        # 스케일별로 mean값 뽑아서 mean_리스트에 넣음 [1클래스 ap 평균, 2클래스 ap 평균, ...]
        for i in range(num_scales):
            if np.any(all_num_gts[:, i] > 0):
                mean_ap.append(all_ap[all_num_gts[:, i] > 0, i].mean())
                mean_f1.append(all_f1[all_num_gts[:, i] > 0, i].mean())
            else:
                mean_ap.append(0.0)
                mean_f1.append(0.0)

    # scale_ranges 없으면 클래스별로 리스트에 넣어서 평균냄
    else:
        aps = []
        f1s = []

        for cls_result in eval_results:
            # 클래스의 gt가 있으면, 각 클래스별 ap, f1값 각각 리스트에 넣음 aps = [1클래스 ap, 2클래스 ap, ...]
            if cls_result['num_gts'] > 0:
                aps.append(cls_result['ap'])
                f1s.append(cls_result['F1'])

        mean_ap = np.array(aps).mean() if aps else 0.0
        mean_f1 = np.array(f1s).mean() if f1s else 0.0

    print_map_summary(
        mean_ap, mean_f1, eval_results, dataset, area_ranges, logger=logger)

    return mean_ap, mean_f1, eval_results
    
    # eval_results에 클래스별로 걊들어가있음

def print_map_summary(mean_ap,
                      mean_f1,
                      results,
                      dataset=None,
                      scale_ranges=None,
                      logger=None):

    """Print mAP and results of each class.

    A table will be printed to show the gts/dets/recall/AP/F1!!!!! of each class and
    the mAP,mF1.

    Args:
        mean_ap (float): Calculated from `eval_map()`.
        mean_f1 (list): Calculated from `eval_map()`. 
            mean_f1 = [weighted_f1, macro_f1]
        results (list[dict]): Calculated from `eval_map()`.
        dataset (list[str] | str | None): Dataset name or dataset classes.
        scale_ranges (list[tuple] | None): Range of scales to be evaluated.
        logger (logging.Logger | str | None): The way to print the mAP
            summary. See `mmcv.utils.print_log()` for details. Default: None.
    """

    if logger == 'silent':
        return

    # scale_ranges 있었는지 확인하는 듯
    if isinstance(results[0]['ap'], np.ndarray):
        num_scales = len(results[0]['ap']) # 클래스 1의 ap?
    else:
        num_scales = 1

    # scale_ranges가 있으면, 계산할때랑 맞는지 확인함
    if scale_ranges is not None:
        assert len(scale_ranges) == num_scales

    num_classes = len(results)

    # 각 값들 (0,0,0)으로 세팅
    recalls = np.zeros((num_scales, num_classes), dtype=np.float32) 
    precisions = np.zeros((num_scales, num_classes), dtype=np.float32) 
    aps = np.zeros((num_scales, num_classes), dtype=np.float32)
    f1s = np.zeros((num_scales, num_classes), dtype=np.float32)
    num_gts = np.zeros((num_scales, num_classes), dtype=int)
    for i, cls_result in enumerate(results):
        #클래스별로 꺼내는데, recall값이 있으면,
        if cls_result['recall'].size > 0:
            recalls[:, i] = np.array(cls_result['recall'], ndmin=2)[:, -1]
            precisions[:, i] = np.array(cls_result['precision'], ndmin=2)[:, -1]
        aps[:, i] = cls_result['ap']
        f1s[:, i] = cls_result['F1']
        num_gts[:, i] = cls_result['num_gts']

    if dataset is None:
        label_names = [str(i) for i in range(num_classes)]
    else:
        label_names = dataset

    if not isinstance(mean_ap, list):
        mean_ap = [mean_ap]

    if not isinstance(mean_f1, list):
        mean_f1 = [mean_f1]

    header = ['class', 'gts', 'dets', 'recall', 'precision', 'ap', 'F1']
    for i in range(num_scales):
        if scale_ranges is not None:
            print_log(f'Scale range {scale_ranges[i]}', logger=logger)
        table_data = [header]
        for j in range(num_classes):
            row_data = [
                label_names[j], num_gts[i, j], results[j]['num_dets'],
                f'{recalls[i, j]:.3f}', f'{precisions[i, j]:.3f}', f'{aps[i, j]:.3f}', f'{f1s[i, j]:.3f}'
            ]
            table_data.append(row_data)
        append_data = ['mAP, mF1', '', '', '','', f'{mean_ap[i]:.3f}', f'{mean_f1[i]:.3f}']
        table_data.append(append_data)
        table = AsciiTable(table_data)
        table.inner_footing_row_border = True
        print_log('\\n' + table.table, logger=logger)

'''

eval_map_path = '/content/mmrotate/mmrotate/core/evaluation/eval_map.py'

f = open(eval_map_path, "w")
f.write(eval_map)
f.close()

## model

In [ ]:
model_config = '''

dataset_type = 'FAIR1MDataset'
data_root = '/content/mmrotate/data/split_ss_dota/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='RResize', img_scale=(1024, 1024), multiscale_mode='range'),###################### <---------------- multiscale_mode='range' 추가완료 #####################
    dict(type='RRandomCrop', crop_size=(512, 512)), ###################### <---------------- RRandomCrop추가완료 #####################
    dict(
        type='RRandomFlip',
        flip_ratio=[0.25, 0.25, 0.25],
        direction=['horizontal', 'vertical', 'diagonal'],
        version='le90'),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1024, 1024),
        flip=False,
        transforms=[
            dict(type='RResize'),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='Pad', size_divisor=32),
            dict(type='DefaultFormatBundle'),
            dict(type='Collect', keys=['img'])
        ])
]
data = dict(
    samples_per_gpu=2,
    workers_per_gpu=2,
    train=dict(
        type='FAIR1MDataset',
        ann_file='/content/mmrotate/data/split_ss_dota/train/annfiles/',
        img_prefix='/content/mmrotate/data/split_ss_dota/train/images/',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(type='LoadAnnotations', with_bbox=True),
            dict(type='RResize', img_scale=(1024, 1024), multiscale_mode='range'),
            dict(type='RRandomCrop', crop_size=(512, 512)),
            dict(
                type='RRandomFlip',
                flip_ratio=[0.25, 0.25, 0.25],
                direction=['horizontal', 'vertical', 'diagonal'],
                version='le90'),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='Pad', size_divisor=32),
            dict(type='DefaultFormatBundle'),
            dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
        ],
        version='le90'),
    val=dict(
        type='FAIR1MDataset',
        ann_file='/content/mmrotate/data/split_ss_dota/val/annfiles/',
        img_prefix='/content/mmrotate/data/split_ss_dota/val/images/',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(1024, 1024),
                flip=False,
                transforms=[
                    dict(type='RResize'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='DefaultFormatBundle'),
                    dict(type='Collect', keys=['img'])
                ])
        ],
        version='le90'),
    test=dict(
        type='FAIR1MDataset',
        ann_file='/content/mmrotate/data/split_ss_dota/val/annfiles/',
        img_prefix='/content/mmrotate/data/split_ss_dota/val/images/',
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(1024, 1024),
                flip=False,
                transforms=[
                    dict(type='RResize'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='DefaultFormatBundle'),
                    dict(type='Collect', keys=['img'])
                ])
        ],
        version='le90'))
evaluation = dict(interval=1, metric='mAP' and 'F1Score')
optimizer = dict(type='SGD', lr=0.005, momentum=0.9, weight_decay=0.0001)
optimizer_config = dict(grad_clip=dict(max_norm=35, norm_type=2))
lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=500,
    warmup_ratio=0.3333333333333333,
    step=[8, 11])
runner = dict(type='EpochBasedRunner', max_epochs=12)
checkpoint_config = dict(interval=1)
log_config = dict(
    interval=50,
    hooks=[
        dict(type='TextLoggerHook', interval=50),
        dict(type='WandbLoggerHook',interval=100,
            init_kwargs=dict(
                project='SIA_Project',
                entity = 'jhgsia',
                name = 'ratios_[0.25, 0.5, 1.0, 1.5, 2.0, 4.0],scales=[2, 4, 8]' ###################### <---------------- 여기서 프로젝트에 들어가는 이름 설정#####################
            ),
            )
    ])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = None
workflow = [('train', 1)]
opencv_num_threads = 0
mp_start_method = 'fork'
angle_version = 'le90'
model = dict(
    type='OrientedRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='OrientedRPNHead',
        in_channels=256,
        feat_channels=256,
        version='le90',
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[2, 4, 8],
            ratios=[0.25, 0.5, 1.0, 1.5, 2.0, 4.0],     ###################### <------------여기 자기가 맡은 ratio로 수정 ##################
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='MidpointOffsetCoder',
            angle_range='le90',
            target_means=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0, 0.5, 0.5]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(
            type='SmoothL1Loss', beta=0.1111111111111111, loss_weight=1.0)),
    roi_head=dict(
        type='OrientedStandardRoIHead',
        bbox_roi_extractor=dict(
            type='RotatedSingleRoIExtractor',
            roi_layer=dict(
                type='RoIAlignRotated',
                out_size=7,
                sample_num=2,
                clockwise=True),
            out_channels=256,
            featmap_strides=[4, 8, 16, 32]),
        bbox_head=dict(
            type='RotatedShared2FCBBoxHead',
            in_channels=256,
            fc_out_channels=1024,
            roi_feat_size=7,
            num_classes=3,
            bbox_coder=dict(
                type='DeltaXYWHAOBBoxCoder',
                angle_range='le90',
                norm_factor=None,
                edge_swap=True,
                proj_xy=True,
                target_means=(0.0, 0.0, 0.0, 0.0, 0.0),
                target_stds=(0.1, 0.1, 0.2, 0.2, 0.1)),
            reg_class_agnostic=True,
            loss_cls=dict(
                type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0),
            loss_bbox=dict(type='SmoothL1Loss', beta=1.0, loss_weight=1.0))),
    train_cfg=dict(
        rpn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.7,
                neg_iou_thr=0.3,
                min_pos_iou=0.3,
                match_low_quality=True,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=256,
                pos_fraction=0.5,
                neg_pos_ub=-1,
                add_gt_as_proposals=False),
            allowed_border=0,
            pos_weight=-1,
            debug=False),
        rpn_proposal=dict(
            nms_pre=2000,
            max_per_img=2000,
            nms=dict(type='nms', iou_threshold=0.8),
            min_bbox_size=0),
        rcnn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.5,
                neg_iou_thr=0.5,
                min_pos_iou=0.5,
                match_low_quality=False,
                iou_calculator=dict(type='RBboxOverlaps2D'),
                ignore_iof_thr=-1),
            sampler=dict(
                type='RRandomSampler',
                num=512,
                pos_fraction=0.25,
                neg_pos_ub=-1,
                add_gt_as_proposals=True),
            pos_weight=-1,
            debug=False)),
    test_cfg=dict(
        rpn=dict(
            nms_pre=2000,
            max_per_img=2000,
            nms=dict(type='nms', iou_threshold=0.8),
            min_bbox_size=0),
        rcnn=dict(
            nms_pre=2000,
            min_bbox_size=0,
            score_thr=0.05,
            nms=dict(iou_thr=0.1),
            max_per_img=2000)))

'''

model_config_path = '/content/mmrotate/configs/oriented_rcnn/oriented_rcnn_r50_fpn_1x_dota_le90.py'

f = open(model_config_path, "w")
f.write(model_config)
f.close()

#Train!

In [1]:
%cd /content/mmrotate

[Errno 2] No such file or directory: '/content/mmrotate'
/content


In [2]:
!python ./tools/train.py ./configs/oriented_rcnn/oriented_rcnn_r50_fpn_1x_dota_le90.py

python3: can't open file './tools/train.py': [Errno 2] No such file or directory
